In [34]:
import os
import requests
import alpaca_trade_api as tradeapi
import pandas as pd
from dotenv import load_dotenv


In [35]:
load_dotenv()

True

In [36]:
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')
print(type(alpaca_api_key))

<class 'str'>


In [37]:
alpaca=tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version='v2'
)

In [38]:
sectors_top5_dict = {
    'energy' : ['XOM', 'CVX', 'SHEL', 'TTE', 'COP'],
    'materials' : ['BHP', 'LIN', 'RIO', 'VALE', 'APD'],
    'industrials' : ['UPS', 'RTX', 'HON', 'CAT', 'UNP'],
    'utilities' : ['NEE', 'DUK', 'SO', 'D', 'SRE'],
    'healthcare' : ['UNH', 'JNJ', 'LLY', 'NVO', 'MRK'],
    'financials' : ['BRK-A', 'V', 'JPM', 'MA', 'BAC'],
    'consumer_discretionary' : ['AMZN', 'TSLA', 'HD', 'BABA', 'TM'],
    'consumer_staples' : ['WMT', 'PG', 'KO', 'PEP', 'COST'],
    'tech' : ['AAPL', 'MSFT', 'TSM', 'NVDA', 'ASML'],
    'communications' : ['GOOG', 'META', 'DIS', 'TMUS', 'CMCSA'],
    'real_estate' : ['PLD', 'AMT', 'EQIX', 'CCI', 'PSA'],
    'saas' : ['DDOG', 'NET', 'ZM', 'CRWD', 'ZS', 'ZI', 'BILL', 'MNDY', 'SNOW']
}

In [39]:
start_date = pd.Timestamp("2020-06-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-06-05", tz="America/New_York").isoformat()
timeframe='1Day'

In [40]:
# Create an empty dictionary to hold the dataframes of data for each sector.
sectors_df_dict = {} 

# Create a separate dataframe for each sector by looping through the keys of sectors_top5_dict.
for sector in sectors_top5_dict:
    sectors_df_dict[sector] = alpaca.get_bars(
        sectors_top5_dict[sector],
        timeframe,
        start = start_date,
        end = end_date
    ).df

# Create an empty dictionary to hold the dataframes of data for each ticker in the sector.
    ticker_dict = {}

# Enter each ticker's data into the ticker_dict dictionary
# where the key is the ticker and the value is the dataframe for that ticker.
    for ticker in sectors_top5_dict[sector]:
        ticker_dict[ticker] = sectors_df_dict[sector][sectors_df_dict[sector]['symbol']==ticker].drop(columns='symbol', axis=1)

# Concatenate the data for each ticker horizontally so each dataframe
# has a second level key with the name of the ticker. 
    ticker_df = pd.concat(ticker_dict.values(), axis=1, keys = sectors_top5_dict[sector])

# Add the resulting dataframe into the sectors_df_dict dictionary with the
# name of the sector as the key.
    sectors_df_dict[sector] = ticker_df

# Concatenate the dataframe for each sector so each has a third level key
# with the name of the sector.
sectors_df = pd.concat(sectors_df_dict.values(), axis = 1, keys = sectors_top5_dict.keys())
display(sectors_df)



energy                                    \
                             XOM                                     
                            open    high      low  close    volume   
timestamp                                                            
2020-06-01 04:00:00+00:00  45.32  46.450  44.8000  46.28  19038651   
2020-06-02 04:00:00+00:00  46.86  47.450  46.7474  47.31  22318978   
2020-06-03 04:00:00+00:00  48.00  49.400  47.9500  49.24  23418047   
2020-06-04 04:00:00+00:00  48.88  49.145  48.1600  49.10  18731391   
2020-06-05 04:00:00+00:00  51.65  53.450  51.6000  53.08  41329508   

                                                                         ...  \
                                                    CVX                  ...   
                          trade_count       vwap   open    high     low  ...   
timestamp                                                                ...   
2020-06-01 04:00:00+00:00      118690  45.903275  90.94   93.85  90.110  ...   
2020-06-02 04:00:00+00:00      121839  47.157322  93.78   95.22  93.285  ...   
2020-06-03 04:00:00+00:00      145141  48.870036  95.97   97.73  95.750  ...   
2020-06-04 04:00:00+00:00      124945  48.794668  96.78   96.89  95.160  ...   
2020-06-05 04:00:00+00:00      243227  52.799124  99.08  102.05  99.050  ...   

                            saas                                               \
                            MNDY                    SNOW                        
                          volume trade_count vwap   open   high    low  close   
timestamp                                                                       
2020-06-01 04:00:00+00:00    NaN         NaN  NaN  23.75  23.75  23.75  23.75   
2020-06-02 04:00:00+00:00    NaN         NaN  NaN  23.75  23.75  23.75  23.75   
2020-06-03 04:00:00+00:00    NaN         NaN  NaN  23.75  23.75  23.75  23.75   
2020-06-04 04:00:00+00:00    NaN         NaN  NaN  23.75  23.75  23.75  23.75   
2020-06-05 04:00:00+00:00    NaN         NaN  NaN  23.75  23.75  23.75  23.75   

                                                   
                                                   
                          volume trade_count vwap  
timestamp                                          
2020-06-01 04:00:00+00:00      0           0  0.0  
2020-06-02 04:00:00+00:00      0           0  0.0  
2020-06-03 04:00:00+00:00      0           0  0.0  
2020-06-04 04:00:00+00:00      0           0  0.0  
2020-06-05 04:00:00+00:00      0           0  0.0  

[5 rows x 448 columns]